In [ ]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

# Connecting Variables

To cause data to flow between two systems in a model, we must connect at
least one output variable from one system to at least one input variable
from the other.  If the variables have units defined, then the framework
will automatically perform the conversion.  We can also connect only part
of an array output to an input by specifying the indices of the entries
that we want.

To connect two variables within a model, use the `connect` function.

```{eval-rst}
    .. automethod:: openmdao.core.group.Group.connect
        :noindex:
```

## Usage

1: Connect an output variable to an input variable, with an automatic unit conversion.  

In [ ]:
import numpy as np
import openmdao.api as om

p = om.Problem()

p.model.set_input_defaults('x', np.ones(5), units='ft')

exec_comp = om.ExecComp('y=sum(x)',
                        x={'val': np.zeros(5), 'units': 'inch'},
                        y={'units': 'inch'})

p.model.add_subsystem('comp1', exec_comp, promotes_inputs=['x'])

p.setup()
p.run_model()

print(p.get_val('x', units='ft'))
print(p.get_val('comp1.x'))
print(p.get_val('comp1.y'))

In [ ]:
from openmdao.utils.assert_utils import assert_near_equal

assert_near_equal(p.get_val('x', units='ft'), np.ones(5))
assert_near_equal(p.get_val('comp1.x'), np.ones(5)*12.)
assert_near_equal(p.get_val('comp1.y'), 60.)

2: Connect one output to many inputs.  

In [ ]:
p = om.Problem()

p.model.add_subsystem('C1', om.ExecComp('y=sum(x)*2.0', x=np.zeros(5)), promotes_inputs=['x'])
p.model.add_subsystem('C2', om.ExecComp('y=sum(x)*4.0', x=np.zeros(5)), promotes_inputs=['x'])
p.model.add_subsystem('C3', om.ExecComp('y=sum(x)*6.0', x=np.zeros(5)), promotes_inputs=['x'])

p.setup()
p.set_val('x', np.ones(5))
p.run_model()

print(p.get_val('C1.y'))
print(p.get_val('C2.y'))
print(p.get_val('C3.y'))

In [ ]:
assert_near_equal(p.get_val('C1.y'), 10.)
assert_near_equal(p.get_val('C2.y'), 20.)
assert_near_equal(p.get_val('C3.y'), 30.)

(connect-with-src-indices)=
3: Connect only part of an array output to an input of a smaller size.

In [ ]:
p = om.Problem()

p.model.add_subsystem('indep', om.IndepVarComp('x', np.ones(5)))
p.model.add_subsystem('C1', om.ExecComp('y=sum(x)*2.0', x=np.zeros(3)))
p.model.add_subsystem('C2', om.ExecComp('y=sum(x)*4.0', x=np.zeros(2)))

# connect C1.x to the first 3 entries of indep.x
p.model.connect('indep.x', 'C1.x', src_indices=[0, 1, 2])

# connect C2.x to the last 2 entries of indep.x
# use -2 (same as 3 in this case) to show that negative indices work.
p.model.connect('indep.x', 'C2.x', src_indices=[-2, 4])

p.setup()
p.run_model()

print(p['C1.x'])
print(p['C1.y'])
print(p['C2.x'])
print(p['C2.y'])

In [ ]:
assert_near_equal(p['C1.x'], np.ones(3))
assert_near_equal(p['C1.y'], 6.)
assert_near_equal(p['C2.x'], np.ones(2))
assert_near_equal(p['C2.y'], 8.)

4: Connect only part of a non-flat array output to a non-flat array input.

In [ ]:
p = om.Problem()

p.model.add_subsystem('indep', om.IndepVarComp('x', np.arange(12).reshape((4, 3))))
p.model.add_subsystem('C1', om.ExecComp('y=sum(x)*2.0', x=np.zeros((2, 2))))

# Connect C1.x to entries (0,0), (-1,1), (2,1), (1,1) of indep.x and give then a 2x2 shape.
# To do this, create 2x2 shaped arrays representing rows and columns. Note that the final
# src_indices is a tuple (not a list or array) containing the rows and cols arrays.
rows = [[0, -1],[2, 1]]
cols = [[0, 1], [1, 1]]
p.model.connect('indep.x', 'C1.x', src_indices=(rows, cols), flat_src_indices=False)

p.setup()
p.run_model()

print(p.get_val('indep.x'))
print(p.get_val('C1.x'))
print(p.get_val('C1.y'))

In [ ]:
assert_near_equal(p.get_val('indep.x'), np.array([[0., 1., 2.],
                                                  [3., 4., 5.],
                                                  [6., 7., 8.],
                                                  [9., 10., 11.]]))
assert_near_equal(p.get_val('C1.x'), np.array([[0., 10.],
                                               [7., 4.]]))
assert_near_equal(p.get_val('C1.y'), 42.)

## Using `om.slicer` to connect non-scalar variables

The `om.slicer` object was introduced to OpenMDAO to make connecting non-scalar variables easier by allowing the `src_indices` to be specified as slices.
This way of thinking about `src_indices` is probably more intuitive for users who are experienced with numpy.
    
The following example connects a 2x2 portion of a 5x5 matrix, starting from indices [2, 3]:

In [ ]:
p = om.Problem()

ivc = p.model.add_subsystem('ivc', om.IndepVarComp())
ivc.add_output('M', val=np.arange(25).reshape((5, 5)))

exec = p.model.add_subsystem('exec', om.ExecComp())

exec.add_expr('A = B', A={'shape': (2, 2)}, B={'shape': (2, 2)})

p.model.connect('ivc.M', 'exec.B', src_indices=om.slicer[2:4, 3:5])

p.setup()

p.run_model()

print('M')
print(p.get_val('ivc.M'))

print('A')
print(p.get_val('exec.A'))

In [ ]:
from openmdao.utils.assert_utils import assert_near_equal

expected = np.array([[13, 14], [18, 19]])

assert_near_equal(p.get_val('exec.A'), expected)

The following example connects the first 3 columns of rows 0, 2, and 3 of `M` to `B`:

In [ ]:
p = om.Problem()

ivc = p.model.add_subsystem('ivc', om.IndepVarComp())
ivc.add_output('M', val=np.arange(25).reshape((5, 5)))

exec = p.model.add_subsystem('exec', om.ExecComp())

exec.add_expr('A = B', A={'shape': (3, 3)}, B={'shape': (3, 3)})

p.model.connect('ivc.M', 'exec.B', src_indices=om.slicer[[0, 2 ,3], :3])

p.setup()

p.run_model()

print('M')
print(p.get_val('ivc.M'))

print('A')
print(p.get_val('exec.A'))

In [ ]:
from openmdao.utils.assert_utils import assert_near_equal

expected = np.array([[0, 1, 2], [10, 11, 12], [15, 16, 17]])

assert_near_equal(p.get_val('exec.A'), expected)

## The meaning of `flat_src_indices` when using `om.slicer`

When `src_indices` are provided as a slicer and `flat_src_indices = True`, OpenMDAO first flattens (in row-major order), the source variable and then applies the slicer.
Consider connecting the 3x3 matrix `M` below to a target 3-vector.
In this first case, we don't flatten the matrix before indexing it (`flat_src_indices=False`), so in this case `om.slicer[:1]` refers to the first element in the first dimension (the first row).

In [ ]:
p = om.Problem()

ivc = p.model.add_subsystem('ivc', om.IndepVarComp())
M_np = np.arange(9)[::-1].reshape((3, 3))
ivc.add_output('M', val=M_np)

exec = p.model.add_subsystem('exec', om.ExecComp())

exec.add_expr('A = B', A={'shape': (1, 3)}, B={'shape': (3,)})

p.model.connect('ivc.M', 'exec.B', src_indices=om.slicer[:1])

p.setup()

p.run_model()

print('M')
print(p.get_val('ivc.M'))
print()
print('A')
print(p.get_val('exec.A'))

In [ ]:
assert_near_equal(p.get_val('exec.A'), p.get_val('ivc.M')[:1])

If, on the other hand, we specify `flat_src_indices=True` in the connection, then we will first flatten M, and pass the first element of the flattened array to `exec.B`.

In [ ]:
p = om.Problem()

ivc = p.model.add_subsystem('ivc', om.IndepVarComp())
ivc.add_output('M', val=np.arange(9)[::-1].reshape((3, 3)))

exec = p.model.add_subsystem('exec', om.ExecComp())

exec.add_expr('A = B', A={'shape': (1,)}, B={'shape': (1,)})

p.model.connect('ivc.M', 'exec.B', src_indices=om.slicer[:1], flat_src_indices=True)

p.setup()

p.run_model()

print('M')
print(p.get_val('ivc.M'))

print('A')
print(p.get_val('exec.A'))

In [ ]:
assert_near_equal(p.get_val('exec.A'), p.get_val('ivc.M')[0, 0])